In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import os


In [2]:


# Define the directory path to your dataset
data_dir = '/kaggle/input/eye-disease-image-dataset/Augmented Dataset/Augmented Dataset/'

# Create an ImageDataGenerator with rescaling and a validation split.
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Use 20% of the data for validation
)

# Training data generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # VGG16 expects 224x224 images
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Validation data generator
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 12997 images belonging to 10 classes.
Found 3245 images belonging to 10 classes.


In [3]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [6]:


for layer in base_model.layers:
    layer.trainable = False


In [7]:

x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [9]:

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10  # Feel free to adjust the number of epochs
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


406/406 ━━━━━━━━━━━━━━━━━━━━ 284s 651ms/step - accuracy: 0.3345 - loss: 1.8929 - val_accuracy: 0.4864 - val_loss: 1.5223
Epoch 2/10
  1/406 ━━━━━━━━━━━━━━━━━━━━ 1:05 162ms/step - accuracy: 0.5000 - loss: 1.5129

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


406/406 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.5000 - loss: 1.5129 - val_accuracy: 0.3846 - val_loss: 1.6211
Epoch 3/10
406/406 ━━━━━━━━━━━━━━━━━━━━ 203s 493ms/step - accuracy: 0.4881 - loss: 1.4433 - val_accuracy: 0.5486 - val_loss: 1.3514
Epoch 4/10
406/406 ━━━━━━━━━━━━━━━━━━━━ 0s 220us/step - accuracy: 0.3438 - loss: 1.5076 - val_accuracy: 0.3846 - val_loss: 1.5751
Epoch 5/10
406/406 ━━━━━━━━━━━━━━━━━━━━ 198s 481ms/step - accuracy: 0.5565 - loss: 1.2617 - val_accuracy: 0.5387 - val_loss: 1.2795
Epoch 6/10
406/406 ━━━━━━━━━━━━━━━━━━━━ 0s 218us/step - accuracy: 0.4688 - loss: 1.3906 - val_accuracy: 0.5385 - val_loss: 1.4809
Epoch 7/10
406/406 ━━━━━━━━━━━━━━━━━━━━ 200s 486ms/step - accuracy: 0.5760 - loss: 1.2051 - val_accuracy: 0.5736 - val_loss: 1.2447
Epoch 8/10
406/406 ━━━━━━━━━━━━━━━━━━━━ 0s 207us/step - accuracy: 0.4062 - loss: 1.6052 - val_accuracy: 0.5385 - val_loss: 0.9978
Epoch 9/10
406/406 ━━━━━━━━━━━━━━━━━━━━ 201s 489ms/step - accuracy: 0.6048 - loss: 1.1241 - val